In [1]:
## This file is meant to be run only once
%matplotlib inline

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
import geopandas as gpd
import os.path
#from PIL import Image
import rasterio
import math

# read in all of your files #TODO make sure these match your file path
#os.chdir("/oak/stanford/groups/omramom/group_members/aminaly/mountain_biodiversity")
os.chdir("/Users/aminaly/Box Sync/mountain_biodiversity")

In [2]:
#### read in and combine all of the wdpa files. ONLY NEEDS TO RUN ONCE

## read in all the wdpa files
wdpa0 = gpd.read_file(os.getcwd() + "/data/WDPA/WDPA_Jun2021_Public_shp/WDPA_Jun2021_Public_shp_0/WDPA_Jun2021_Public_shp-polygons.shp")
wdpa1 = gpd.read_file(os.getcwd() + "/data/WDPA/WDPA_Jun2021_Public_shp/WDPA_Jun2021_Public_shp_1/WDPA_Jun2021_Public_shp-polygons.shp")
wdpa2 = gpd.read_file(os.getcwd() + "/data/WDPA/WDPA_Jun2021_Public_shp/WDPA_Jun2021_Public_shp_2/WDPA_Jun2021_Public_shp-polygons.shp")
wdpa3 = gpd.read_file(os.getcwd() + "/data/WDPA/WDPA_Jun2021_Public_shp/WDPA_Jun2021_Public_shp_0/WDPA_Jun2021_Public_shp-points.shp")
wdpa4 = gpd.read_file(os.getcwd() + "/data/WDPA/WDPA_Jun2021_Public_shp/WDPA_Jun2021_Public_shp_1/WDPA_Jun2021_Public_shp-points.shp")
wdpa5 = gpd.read_file(os.getcwd() + "/data/WDPA/WDPA_Jun2021_Public_shp/WDPA_Jun2021_Public_shp_2/WDPA_Jun2021_Public_shp-points.shp")

#combine the poly and point dataframes
wdpa_poly = gpd.GeoDataFrame(pd.concat([wdpa0, wdpa1, wdpa2]))
wdpa_point = gpd.GeoDataFrame(pd.concat([wdpa3, wdpa4, wdpa5]))

wdpa_poly.crs = {'init': 'epsg:4326', 'no_defs': True}
wdpa_point.crs = {'init': 'epsg:4326', 'no_defs': True}

## filter out marine locations 
wdpa_poly = wdpa_poly[~wdpa_poly['DESIG'].str.contains("Marine")] 
wdpa_point = wdpa_point[~wdpa_point['DESIG'].str.contains("Marine")]

#save these out so we don't have to re-run again
#wdpa_poly.to_file(os.getcwd() + "/data/WDPA/WDPA_Jun2021_Public_shp/WDPA_Jun2021_Public/WDPA_Jun2021_Public_shp-polygons.shp", driver='ESRI Shapefile')
#wdpa_point.to_file(os.getcwd() + "/data/WDPA/WDPA_Jun2021_Public_shp/WDPA_Jun2021_Public/WDPA_Jun2021_Public_shp-points.shp", driver='ESRI Shapefile')

In [ ]:
#### If you don't need to run the above cell, here are the final poly and points to work qith
wdpa_poly = gpd.read_file(os.getcwd() + "/data/WDPA/WDPA_Jun2021_Public_shp/WDPA_Jun2021_Public/WDPA_Jun2021_Public_shp-polygons.shp", driver='ESRI Shapefile')
wdpa_point = gpd.read_file(os.getcwd() + "/data/WDPA/WDPA_Jun2021_Public_shp/WDPA_Jun2021_Public/WDPA_Jun2021_Public_shp-points.shp", driver='ESRI Shapefile')


In [ ]:
#### Take in points and create buffers around them based on reported area

## Add buffers to the point based on their area
wdpa_point.crs = {'init': 'epsg:3763', 'no_defs': True}
wdpa_point.to_crs({'init': 'epsg:3763'})

## calculate the radius
wdpa_point['radius'] = np.sqrt(wdpa_point.REP_AREA / np.pi) / 1000

## remove entries with 0 reported area/radius
wdpa_point.drop(wdpa_point[wdpa_point['radius'] <= 0].index, inplace = True)

## create the buffers
wdpa_point['geometry'] = wdpa_point.geometry.buffer(wdpa_point.radius)

# convert projection back
wdpa_point.crs = {'init': 'epsg:4326', 'no_defs': True}
wdpa_point.to_crs({'init': 'epsg:4326'})

In [ ]:
#### Identify all the overlaps, dissolve, and combine with non-overlaps
overlaps_poly = wdpa_point.intersects(wdpa_poly)

# subset poly and point based on the identified overlaps 
#overlaps_poly = overlaps_poly.reset_index().drop(columns = "index")
overlaps_poly_sh = wdpa_poly[np.array(overlaps_poly)]
nonoverlaps_poly_sh = wdpa_poly[~overlaps_poly]

# combine the overlapping polygons with the points 
overlap_poly_point = gpd.GeoDataFrame(pd.concat([overlaps_poly_sh, wdpa_point]))

#dissolve the overlapping polygons with all the points
dissolved_overlaps = overlap_poly_point.dissolve(by='WDPAID')

# concat new dissolved layer, with the nonoverlapping polygons
wdpa_final = gpd.GeoDataFrame(pd.concat([nonoverlaps_poly_sh, dissolved_overlaps]))

#make sure the CRS is correct
wdpa_final.crs = {'init': 'epsg:4326', 'no_defs': True}
wdpa_final.to_crs({'init': 'epsg:4326'})

#save out
#wdpa_final.to_file(os.getcwd() + "/data/WDPA/WDPA_Jun2021_Public_shp/WDPA_Jun2021_Public/WDPA_Jun2021_Public_flattened.shp", driver='ESRI Shapefile')


In [3]:
## Add buffers to the point based on their area
wdpa_point.crs = {'init': 'epsg:3763', 'no_defs': True}
wdpa_point.to_crs({'init': 'epsg:3763'})

## calculate the radius
wdpa_point['radius'] = np.sqrt(wdpa_point.REP_AREA / np.pi) / 1000

## remove entries with 0 reported area/radius
wdpa_point.drop(wdpa_point[wdpa_point['radius'] <= 0].index, inplace = True)

## create the buffers
wdpa_point['geometry'] = wdpa_point.geometry.buffer(wdpa_point.radius)

# convert projection back
wdpa_point.crs = {'init': 'epsg:4326', 'no_defs': True}
wdpa_point.to_crs({'init': 'epsg:4326'})

wrld_cntries = ['AZE', 'LVA', 'CHE', 'DEU', 'BDI', 'GHA', 'KOR', 'NIC', 'SAU', 'ESP']
wdpa_poly_c = wdpa_poly[wdpa_poly['ISO3'].isin(wrld_cntries)]
wdpa_point_c = wdpa_point[wdpa_point['ISO3'].isin(wrld_cntries)]

overlap_poly_point = gpd.GeoDataFrame(pd.concat([wdpa_poly_c, wdpa_point_c]))
dissolved = overlap_poly_point.dissolve(by='WDPAID')

dissolved.crs = {'init': 'epsg:4326', 'no_defs': True}
dissolved.to_crs({'init': 'epsg:4326'})

dissolved.to_file(os.getcwd() + "/data/WDPA/WDPA_Jun2021_Public_shp/WDPA_Jun2021_Public/clipped_WDPA_Jun2021_Public_flattened.shp", driver='ESRI Shapefile')

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

